In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../data/complaints.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
df['Date'] = pd.to_datetime(df['Date received'],infer_datetime_format=True)
print(df['Date'].min())
print(df['Date'].max())
df.info()

In [ ]:
df_grouped = df.groupby('Company').count()[['Complaint ID']]

In [ ]:
dfT20 = df_grouped.sort_values('Complaint ID', ascending=False).head(20)

In [ ]:
dfT20.insert(loc=1, column='Top 20', value=np.arange(len(dfT20)) +1)
dfT20.rename(columns={'Complaint ID':"Issue Counts"})

In [ ]:
df_issues_dates = df.groupby('Date').count()[['Complaint ID']]
df_issues_dates.info()

In [ ]:
df_issues_dates = df_issues_dates.reset_index()
df_issues_dates.info()

In [ ]:
df_issues_dates = df_issues_dates.rename(columns={'Complaint ID':"Issue Counts"})

In [ ]:
fig = plt.gcf()
fig.set_size_inches(6, 6)
ax = plt.gca()
df_issues_dates.plot(kind='line',x='Date',y='Issue Counts',ax=ax)

In [ ]:
dfT20 = dfT20.reset_index()
dfT20.info()

In [ ]:
dfAT20 = df.loc[df['Company'].isin(dfT20['Company'])] #reduce original data to T20

In [ ]:
def pickCompany(company,df):
    dfC = df.loc[df['Company'] == company]
    dfC_dates = dfC.groupby('Date').count()[['Complaint ID']]
    dfC_dates = dfC_dates.reset_index()
    dfC_dates = dfC_dates.rename(columns={'Complaint ID':company}) 
    return dfC_dates

In [ ]:
def mergeCompanies(dfT,n,df):
    columns = ['Date']
    dfm = pd.DataFrame(columns=columns)
    i=0
    while i<n:
        df1 = pickCompany(dfT['Company'][i], df)
        dfm = pd.merge(dfm, df1, on='Date', how='outer')
        i += 1
    dfm = dfm.fillna(0)
    return dfm  

In [ ]:
nCompanies=20
dfm = mergeCompanies(dfT20,nCompanies,df)
dfm

In [ ]:
cols = list(dfm.columns)
columns = []
colLabels = []

fig = plt.gcf()
fig.set_size_inches(10,10) 

i=1
while i<=nCompanies:
    colLabels.append(cols[i])
    columns.append(dfm[cols[i]])
    plt.plot(dfm['Date'],dfm[cols[i]],label=cols[i])
    i += 1
    
plt.legend()

In [ ]:
fig = plt.gcf()
fig.set_size_inches(10,10) 
plt.stackplot(dfm['Date'],columns,labels=colLabels)
plt.legend(loc='upper left')